# Coletando daos IBGE, Armazenando no BigQuery e Preenchendo PPT

**Vamos divir o processo em duas etapas para facilidar a Didatica**
- Primeira Etapa: Coleta de dados do IBGE e Armazenamento no BigQuery
- Segunda Etapa: Consumo dos dados do BigQuery e Prenchimento do PPT


### Segunda Etapa: Consumo dos dados do BigQuery e Prenchimento do PPT
Nesta etapa vamos consultar os dados armazenados no BigQuery e preencher o PPT. 

**Importa Libs**

In [12]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas_gbq
from pptx import Presentation
from pptx.chart.data import CategoryChartData
#import pandas as pd


#### Abrindo PPT

In [13]:
pres = Presentation("arqs/pessoasMunicipio.pptx")

#### Seleciona Slide a ser alterado

In [14]:
slides = [slide for slide in pres.slides] 
slide2 = slides[1]

In [15]:
uf = 'SP' 
titulo_grafico = [objeto for objeto in slide2.shapes if objeto.has_text_frame]
titulo_grafico[0].text = str(titulo_grafico[0].text).format(uf = uf)

### Autenticando no Google Cloud
**Neste ponto iremos utilizar a chave de serviço do GCP**

**Obs:** Caso não saiba gerar a chave pode utilizar o link a seguir:
- https://medium.com/@alexandretavares.bigdata/criando-uma-conta-de-servi%C3%A7o-no-gcp-eeac59f3cbcb

In [16]:
# Informe aqui o caminho onde a chave estará armazenada
# Recomendamos que guarde esta chave em um cofre digital para maior segurança
pathCredencial = "key/dswdemo.json" 

credentials = service_account.Credentials.from_service_account_file(
    pathCredencial,
    scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

#### Busca dados do Municipio informado no BigQuery

In [18]:
QUERY = ( f'''
              DECLARE anos STRING;
              SET anos = (
                SELECT 
                  CONCAT('("', STRING_AGG(DISTINCT ano, '", "'), '")'),
                FROM `raw.tb_nat_populacao_municipio`
              );
              EXECUTE IMMEDIATE format("""
                SELECT * FROM
                (
                  SELECT 
                    municipio,
                    ano,
                    valor
                  FROM `raw.tb_nat_populacao_municipio`
                  where uf = "{uf}"
                )
                PIVOT
                (
                  sum(valor) as ano
                  FOR ano in %s
                )
                ORDER BY municipio ASC
              """, anos)
          '''
    )
df = pandas_gbq.read_gbq(QUERY, credentials=credentials)
df


Downloading: 100%|██████████| 645/645 [00:00<00:00, 1889.65rows/s]


,municipio,ano_2019,ano_2020,ano_2021
0,Adamantina,35068,35111,35153
1,Adolfo,3562,3554,3545
2,Aguaí,36305,36648,36981
3,Agudos,37214,37401,37582
4,Alambari,6025,6129,6231
...,...,...,...,...
640,Águas de São Pedro,3451,3521,3588
641,Álvares Florence,3679,3647,3616
642,Álvares Machado,24915,24998,25078
643,Álvaro de Carvalho,5227,5274,5320


#### Preenche grafico do Slide 2 com dados do BigQuery

In [19]:
slide2_charts = [shape for shape in slide2.shapes if shape.has_chart]
bar_chart = [shape.chart for shape in slide2_charts ]
chart_data  = CategoryChartData()

colunas = [x for x in df.columns]
colunas.pop(0)

chart_data.categories = [str(x) for x in df['municipio']]
for x in colunas:
    chart_data.add_series(x.split('_')[1], [qtdPessoas for qtdPessoas in df[x]])
bar_chart[0].replace_data(chart_data)
bar_chart[0].chart_title.text_frame.text = "População por Municipio"




#### Preenche Titulo do Slide 3

In [20]:
slide3 = slides[2]
titulo_grafico = [objeto for objeto in slide3.shapes if objeto.has_text_frame]
titulo_grafico[0].text = str(titulo_grafico[0].text).format(uf = uf)

#### Consulta dados do Estado no BigQuery

In [21]:
QUERY2 = ( f'''
DECLARE anos STRING;
SET anos = (
  SELECT 
    CONCAT('("', STRING_AGG(DISTINCT ano, '", "'), '")'),
  FROM `raw.tb_nat_populacao_uf`
);
EXECUTE IMMEDIATE format("""
SELECT * FROM
(
  SELECT 
    uf,
    ano,
    valor
  FROM `raw.tb_nat_populacao_uf`
  where uf = "{uf}"
)
PIVOT
(
  sum(valor) AS pessoas
  FOR ano in %s
)
ORDER BY uf ASC
""", anos)
          '''
    )
df = pandas_gbq.read_gbq(QUERY2, credentials=credentials)
df


Downloading: 100%|██████████| 1/1 [00:00<00:00,  2.97rows/s]


,uf,pessoas_2019,pessoas_2020,pessoas_2021
0,SP,45919049,46289333,46649132


#### Preenche Grafico do Slide 2 com dados do Estado Selecionado

In [22]:
slide3_charts = [shape for shape in slide3.shapes if shape.has_chart]
bar_chart2 = [shape.chart for shape in slide3_charts ]
chart_data2  = CategoryChartData()

colunas = [x for x in df.columns]
colunas.pop(0)
chart_data2.categories = [str(x) for x in df['uf']]
for x in colunas:
    chart_data2.add_series(x.split('_')[1], [qtdPessoas for qtdPessoas in df[x]])
bar_chart2[0].replace_data(chart_data2)
bar_chart2[0].chart_title.text_frame.text = "População por Estado"


#### Salva PPT

In [23]:

pres.save('arqs/pessoasMinicipios_Preenchido.pptx')